In [23]:
import pysolr
import pandas as pd

# Load noteevents.csv file
df = pd.read_csv('C:/Content/Mimicdb/noteevents.csv')

# Load admissions.csv file
df_pats = pd.read_csv('C:/Content/Mimicdb/admissions.csv')

# Load DIAGNOSES_ICD file
df_icd = pd.read_csv('C:/Content/Mimicdb/DIAGNOSES_ICD.csv')

# Filter only 'Discharge summary' notes
df = df[df['CATEGORY'] == 'Discharge summary']


C:\Users\91903\AppData\Local\Temp\ipykernel_772\3343117998.py:8: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Content/Mimicdb/noteevents.csv')


In [25]:
# Index the files to get needed fields
docs=[]
for index, row in df.iterrows():  
    hadm_id = int(row['HADM_ID'])
    if hadm_id not in df_pats['HADM_ID'].values:
    # handle case when ROW_ID is not present in df_pats
        expire_flag = 3  # set a default value
    else:
        expire_flag = df_pats.loc[df_pats['HADM_ID'] == hadm_id, 'HOSPITAL_EXPIRE_FLAG'].values[0]

    if hadm_id not in df_icd['HADM_ID'].values:
    # handle case when ROW_ID is not present in df_pats
        icd_codes = ['None']  # set a default value
    else:
        icd_codes = []
        for icd_index in df_icd[df_icd['HADM_ID'] == hadm_id].index:
            icd_code = df_icd.loc[icd_index, 'ICD9_CODE']
            icd_codes.append(str (icd_code))

    icd_str=' '.join(icd_codes)
    doc = {
        'id': row['ROW_ID'],
        'hadm_id': row['HADM_ID'],
        'chartdate': row['CHARTDATE'],
        'category': row['CATEGORY'],
         'icd_codes': str(icd_str) ,
        'expiry_flag': str(expire_flag),
        'text': row['TEXT']
        
    }
    docs.append(doc)

In [26]:
# Connect to Solr core

solr = pysolr.Solr('http://localhost:8983/solr/mimic3/', timeout=100)
# Add documents to Solr index
solr.add(docs)

# Commit changes to Solr index
solr.commit()

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">2804</int>\n</lst>\n</response>\n'